In [1]:
import boto3
import gzip
import json
import pandas as pd
from collections import Counter

from IPython.display import display

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## 1. Load and inspect JSON file

In [3]:
# Open the JSON dataset on s3 bucket
s3_client = boto3.client('s3')
BUCKET = 'nutriflow-pipeline'
KEY = 'phase-a/raw/openfoodfacts-products.jsonl.gz'

obj = s3_client.get_object(Bucket=BUCKET, Key=KEY)

First, I check how many valid records of each grade we have in a chunk of dataset

In [4]:
# Specify the required features to extract
numeric_fields = ["energy_100g",
                    "carbohydrates_100g", "fat_100g",
                    "saturated-fat_100g", "sugars_100g",
                    "salt_100g", "proteins_100g"]

In [5]:
# Extract the data and count valid data for each class
counts = Counter()
valid_recs = 0
Max_lines = 10000

# Stream, filter and count
with gzip.GzipFile(fileobj=obj['Body']) as gz:
    for raw_bytes in gz:
        line = raw_bytes.decode('utf-8')
        item = json.loads(line)
        
        grade = item.get('nutrition_grade_fr')
        nutr = item.get('nutriments', {})
        # Filter only valid data with all numeric fields
        if not grade or any(nutr.get(field) is None for field in numeric_fields):
            continue
        
        counts[grade] += 1
        valid_recs += 1
        if valid_recs >= Max_lines:
            break

print(f"Processed {valid_recs} valid records")
print(f"Nutri-Score distribution:", dict(counts))

Processed 10000 valid records
Nutri-Score distribution: {'e': 3568, 'd': 1738, 'unknown': 636, 'c': 1191, 'b': 772, 'a': 2067, 'not-applicable': 28}


There are records with invalid nutrition score, so I did the counting again based on only valid scores.

In [8]:
obj = s3_client.get_object(Bucket=BUCKET, Key=KEY)
counts = Counter()
valid_recs = 0
Max_lines = 100_000

# Stream, filter and count
with gzip.GzipFile(fileobj=obj['Body']) as gz:
    for raw_bytes in gz:
        line = raw_bytes.decode('utf-8')
        item = json.loads(line)
        
        grade = item.get('nutrition_grade_fr')
        # Filter only valid nutrition scores
        if grade not in ('a', 'b', 'c', 'd', 'e'):
            continue
            
        nutr = item.get('nutriments', {})
        # Filter only valid data with all numeric fields
        if not grade or any(nutr.get(field) is None for field in numeric_fields):
            continue
        
        counts[grade] += 1
        valid_recs += 1
        if valid_recs >= Max_lines:
            break

print(f"Processed {valid_recs} valid records")
print(f"Nutri-Score distribution:", dict(counts))

Processed 100000 valid records
Nutri-Score distribution: {'e': 33508, 'd': 23706, 'c': 15626, 'b': 8300, 'a': 18860}


In [ ]:

core_fields = [
    "code",  
    "nutrition_grade_fr", 
    "nutriments.energy_100g",
    "nutriments.carbohydrates_100g", "nutriments.fat_100g",
    "nutriments.saturated-fat_100g", "nutriments.sugars_100g",
    "nutriments.salt_100g", "nutriments.proteins_100g"
]

#  Function to read json file and extract all core fields and save as a datafram 
def json_to_dataframe(file_path, fields, max_lines=None):
    records = []
    
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if max_lines and i >= max_lines:
                break
            if line.strip():
                item = json.loads(line)
                record = {
                    field.split('.')[-1]: extract_value(item, field)
                    for field in fields
                }
                records.append(record)
    return pd.DataFrame(records)

In [4]:
file_path = "../data/openfoodfacts-products.jsonl.gz"
df = json_to_dataframe(file_path, core_fields, max_lines=500_000)

In [8]:
df.shape

(500000, 13)

In [9]:
df.head()

,code,product_name,brands,categories,ingredients_text,nutrition_grade_fr,energy_100g,carbohydrates_100g,fat_100g,saturated-fat_100g,sugars_100g,salt_100g,proteins_100g
0,0000101209159,Véritable pâte à tartiner noisettes chocolat noir,Bovetti,"Petit-déjeuners,Produits à tartiner,Produits à...",,e,2582.0,36.00,48.0,10.0,32.0,0.01000,8.00
1,0000105000011,Chamomile Herbal Tea,Lagg's,null,CHAMOMILE FLOWERS.,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000105000042,"Lagg's, herbal tea, peppermint",Lagg's,"Plant-based foods and beverages, Beverages, Ho...",Peppermint.,unknown,0.0,1.47,0.0,NaN,NaN,0.01000,0.00
3,0000105000059,Linden Flowers Tea,Lagg's,"Beverages and beverages preparations, Plant-ba...",LINDEN FLOWERS.,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000105000073,"Herbal Tea, Hibiscus",Lagg's,None,Hibiscus flowers.,unknown,1117.0,60.00,0.0,NaN,NaN,0.33782,66.67


In [15]:
output_dir = "../data/clean_data"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "chunk1.parquet")
df.to_parquet(output_path, index=False)

In [16]:
df.isna().sum()

code                      0
product_name           3787
brands                49802
categories            75311
ingredients_text      90434
nutrition_grade_fr        3
energy_100g           41747
carbohydrates_100g    43952
fat_100g              43964
saturated-fat_100g    65989
sugars_100g           53260
salt_100g             54142
proteins_100g         42801
dtype: int64

In [18]:
df['nutrition_grade_fr'].value_counts()

nutrition_grade_fr
unknown           136690
e                 106030
d                  87945
c                  67752
a                  51148
b                  34498
not-applicable     15934
Name: count, dtype: int64